In [178]:
#Añadiendo los paquetes necesarios
using Pkg
Pkg.add("GLM")
Pkg.add("DataFrames")
Pkg.add("DelimitedFiles")
Pkg.add("RData")
Pkg.add("Random")
Pkg.add("Lasso")
Pkg.add("CodecBzip2")
Pkg.add("DataStructures")
Pkg.add("NamedArrays")
Pkg.add("PrettyTables")
Pkg.add("GLMNet")


   Resolving package versions...
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Project.

In [179]:
using GLM, DataFrames, Random, RData, Lasso, LinearAlgebra, Statistics, DataStructures, NamedArrays, PrettyTables, GLMNet
import CodecBzip2

[ Info: Precompiling GLMNet [8d5ece8b-de18-5317-b113-243142960cc6] (cache misses: wrong dep version loaded (2))


In [2]:
#Carpeta Inicio
pwd()

"C:\\Users\\Alvaro\\Documents\\GitHub\\CausalAI-Course\\labs\\replication_1"

In [84]:
#1. Reading the Data
Data = load("wage2015_subsample_inference.Rdata")
Data = Data["data"]

Row,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Cat…,Cat…,Cat…,Cat…
1,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600,11,8370,18
2,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050,10,5070,9
3,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260,19,770,4
4,13.9423,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420,1,6990,12
5,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015,6,9470,22
6,11.7308,2.46222,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,1650,5,7460,14
7,19.2308,2.95651,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,5120,17,7280,14
8,19.2308,2.95651,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,5240,17,5680,9
9,12.0,2.48491,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,4040,13,8590,19


In [85]:
#2. Setting the Alpha Vector
Alpha_Vector = [0.1, 0.2,0.3,0.4,0.5]



5-element Vector{Float64}:
 0.1
 0.2
 0.3
 0.4
 0.5

In [426]:
#3. Shuffle and Folds Generation
Random.seed!(123)  # Set a random seed for reproducibility
Data_Shuffled = shuffle!(Data)
k_folds = 10
Fold_Size = Int.(Total_Observations / k_folds)
Fold_List = []

for number in 1:k_folds
    Fold_Generation = (Data_Shuffled[1+Fold_Size*(number-1):Fold_Size*(number),:])
    push!(Fold_List, Fold_Generation)
end

Validation_fold = Fold_List[10]
Training_Folds = Fold_List[1:9,:]


9×1 Matrix{Any}:
 515×20 DataFrame
 Row │ wage      lwage    sex      shs      hsg      scl      clg      ad      ⋯
     │ Float64   Float64  Float64  Float64  Float64  Float64  Float64  Float64 ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 16.8279   2.82304      0.0      0.0      1.0      0.0      0.0      0.0 ⋯
   2 │ 12.0192   2.48651      0.0      0.0      0.0      1.0      0.0      0.0
   3 │ 12.9808   2.56347      1.0      0.0      1.0      0.0      0.0      0.0
   4 │ 14.9038   2.70162      0.0      0.0      1.0      0.0      0.0      0.0
   5 │ 23.0769   3.13883      0.0      0.0      0.0      0.0      0.0      1.0 ⋯
   6 │ 23.0769   3.13883      1.0      0.0      0.0      0.0      1.0      0.0
   7 │ 48.0769   3.8728       0.0      0.0      0.0      0.0      0.0      1.0
   8 │ 15.3846   2.73337      1.0      0.0      1.0      0.0      0.0      0.0
   9 │ 21.6346   3.07429      0.0      0.0      0.0      1.0      0.0      0.0 ⋯
  10 

In [427]:
Validation_fold

Row,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Cat…,Cat…,Cat…,Cat…
1,4.80769,1.57022,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,17.5,3.0625,5.35938,9.37891,8030,21,2590,6
2,38.4615,3.64966,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,16.0,2.56,4.096,6.5536,1020,3,7380,14
3,21.6346,3.07429,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,34.0,11.56,39.304,133.634,7210,20,6290,10
4,21.1538,3.05182,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.04,0.008,0.0016,910,2,6890,12
5,9.61538,2.26336,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,1.0,1.0,1.0,3850,12,9470,22
6,11.5385,2.44569,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,39.5,15.6025,61.6299,243.438,4010,13,7860,17
7,13.9423,2.63493,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,3.5,0.1225,0.042875,0.0150063,4700,16,5580,9
8,17.5,2.8622,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,3.0,0.09,0.027,0.0081,1640,5,9170,21
9,19.2308,2.95651,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,12.0,1.44,1.728,2.0736,4700,16,4870,9


In [428]:
#4. Lasso Function

function lasso_implementation(dataframe, tested, alpha_value)
    Y = dataframe[!,"wage"]
    Y = DataFrame([Y],[:Y])
    D = dataframe[!,"sex"]
    D = DataFrame([D],[:D])
    W = select(dataframe, Not(["lwage","wage", "sex","occ","occ2","ind","ind2"]))
    data = [Y D W]
    lasso_model = fit(LassoModel, term(:Y) ~  sum(term.(names(data[!, Not(["Y", "D"])]))), data; α = alpha_value)
    Y = tested[!,"wage"]
    Y = DataFrame([Y],[:Y])
    D = tested[!,"sex"]
    D = DataFrame([D],[:D])
    W = select(tested, Not(["lwage","wage", "sex","occ","occ2","ind","ind2"]))
    data_tested = [D W]
    Y_predict = predict(lasso_model, data_tested)
    Y = tested[!,"wage"]
    MSE = mean((Y_predict .- Y).^2)
    return MSE, lasso_model
end





lasso_implementation (generic function with 3 methods)

In [429]:
#5. Cross Validation
MSE_Averages = []
for alpha in Alpha_Vector
    MSE_List = []
    for i in 1:k_folds-1
        tested_fold = Training_Folds[i,:][1]
        Current_Folds = [Training_Folds[z] for z in 1:length(Training_Folds) if !(z == i)]
        Combined_dataframe = vcat(Current_Folds...)
        MSE_current = lasso_implementation(Combined_dataframe,tested_fold,alpha)
        push!(MSE_List, MSE_current[1])
    end
    MSE_Mean = mean(MSE_List)
    println("MSE for this alpha ",alpha,": ",MSE_Mean)
    Alpha_MSE = [MSE_Mean, alpha]
    push!(MSE_Averages, Alpha_MSE)
end



MSE for this alpha 0.1: 424.72350669362817
MSE for this alpha 0.2: 424.70508205301144
MSE for this alpha 0.3: 424.6315179577322
MSE for this alpha 0.4: 424.6616546147978
MSE for this alpha 0.5: 424.74398168510965


In [430]:
#6. Selecting Optimal Alpha

MSE_Averages_Final = []
for i in MSE_Averages
    push!(MSE_Averages_Final, i[1])
end
min_value = minimum(MSE_Averages_Final)
index = findfirst(x -> x == min_value, MSE_Averages_Final)

Optimal_Alpha = MSE_Averages[index][2]
println("Optimal Alpha: ",Optimal_Alpha)
println("Optimal MSE: ",MSE_Averages[index][1])

Optimal Alpha: 0.3
Optimal MSE: 424.6315179577322


In [431]:
#7. Final Training Set
Combined_dataframe = vcat(Training_Folds...)
Last_Alpha = lasso_implementation(Combined_dataframe, Validation_fold, Optimal_Alpha)
MSE = Last_Alpha[1]
println("Final MSE: ", MSE)


Final MSE: 255.46867963689238


In [432]:
#8. Plotting


